# Imports

Import the required libraries.

In [4]:
import re
from typing import Dict, List, Optional, Text, Tuple
import matplotlib.pyplot as plt
from matplotlib import colors

import tensorflow as tf

# Load the dataset

Enter the Unix glob file pattern of the data files.

Here we load the training data. All the data are stored in TensorFlow Record files.
Replace 'train' with 'eval' or 'test' to load the evaluation or testing data, respectively.

In [5]:
file_pattern = '/kaggle/input/next-day-wildfire-spread/next_day_wildfire_spread_train*'
#file_pattern_test = '/kaggle/input/next-day-wildfire-spread/next_day_wildfire_spread_test*'

Run the following three cells to define the required library functions for loading the data.

The first cell defines the name of the variables in the input files and the corrresponding data statistics. The statistics can be used for preprocessing the data. 

In [6]:
"""Constants for the data reader."""

INPUT_FEATURES = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph', 
                  'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask']

OUTPUT_FEATURES = ['FireMask', ]

# Data statistics 
# For each variable, the statistics are ordered in the form:
# (min_clip, max_clip, mean, standard deviation)
DATA_STATS = {
    # Elevation in m.
    # 0.1 percentile, 99.9 percentile
    'elevation': (0.0, 3141.0, 657.3003, 649.0147),
    
    # Drought Index (Palmer Drought Severity Index)
    # 0.1 percentile, 99.9 percentile
    'pdsi': (-6.12974870967865, 7.876040384292651, -0.0052714925, 2.6823447),
    
    #Vegetation index (times 10,000 maybe, since it's supposed to be b/w -1 and 1?)
    'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677),  # min, max
   
    # Precipitation in mm.
    # Negative values do not make sense, so min is set to 0.
    # 0., 99.9 percentile
    'pr': (0.0, 44.53038024902344, 1.7398051, 4.482833),
   
    # Specific humidity.
    # Negative values do not make sense, so min is set to 0.
    # The range of specific humidity is up to 100% so max is 1.
    'sph': (0., 1., 0.0071658953, 0.0042835088),
    
    # Wind direction in degrees clockwise from north.
    # Thus min set to 0 and max set to 360.
    'th': (0., 360.0, 190.32976, 72.59854),
    
    # Min/max temperature in Kelvin.
    
    #Min temp
    # -20 degree C, 99.9 percentile
    'tmmn': (253.15, 298.94891357421875, 281.08768, 8.982386),
    
    #Max temp
    # -20 degree C, 99.9 percentile
    'tmmx': (253.15, 315.09228515625, 295.17383, 9.815496),
    
    # Wind speed in m/s.
    # Negative values do not make sense, given there is a wind direction.
    # 0., 99.9 percentile
    'vs': (0.0, 10.024310074806237, 3.8500874, 1.4109988),
    
    # NFDRS fire danger index energy release component expressed in BTU's per
    # square foot.
    # Negative values do not make sense. Thus min set to zero.
    # 0., 99.9 percentile
    'erc': (0.0, 106.24891662597656, 37.326267, 20.846027),
    
    # Population density
    # min, 99.9 percentile
    'population': (0., 2534.06298828125, 25.531384, 154.72331),
    
    # We don't want to normalize the FireMasks.
    # 1 indicates fire, 0 no fire, -1 unlabeled data
    'PrevFireMask': (-1., 1., 0., 1.),
    'FireMask': (-1., 1., 0., 1.)
}

The following cell defines cropping functions for extracting regions of the desired size from the input data.

In [7]:
"""Library of common functions used in deep learning neural networks.
"""
#YOU PROBABLY WILL NOT USE THESE.

def random_crop_input_and_output_images(
    input_img: tf.Tensor,
    output_img: tf.Tensor,
    sample_size: int,
    num_in_channels: int,
    num_out_channels: int,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Randomly axis-align crop input and output image tensors.

  Args:
    input_img: tensor with dimensions HWC.
    output_img: tensor with dimensions HWC.
    sample_size: side length (square) to crop to.
    num_in_channels: number of channels in input_img.
    num_out_channels: number of channels in output_img.
  Returns:
    input_img: tensor with dimensions HWC.
    output_img: tensor with dimensions HWC.
  """
  combined = tf.concat([input_img, output_img], axis=2)
  combined = tf.image.random_crop(
      combined,
      [sample_size, sample_size, num_in_channels + num_out_channels])
  input_img = combined[:, :, 0:num_in_channels]
  output_img = combined[:, :, -num_out_channels:]
  return input_img, output_img


def center_crop_input_and_output_images(
    input_img: tf.Tensor,
    output_img: tf.Tensor,
    sample_size: int,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Center crops input and output image tensors.

  Args:
    input_img: tensor with dimensions HWC.
    output_img: tensor with dimensions HWC.
    sample_size: side length (square) to crop to.
  Returns:
    input_img: tensor with dimensions HWC.
    output_img: tensor with dimensions HWC.
  """
  central_fraction = sample_size / input_img.shape[0]
  input_img = tf.image.central_crop(input_img, central_fraction)
  output_img = tf.image.central_crop(output_img, central_fraction)
  return input_img, output_img

The following cell provides code for parsing the contents of the TensorFlow Record files. In addition to loading the data, it also offers functions for various preprocessing operations, such as clipping, rescaling, or normalizing the data.  

In [8]:
"""Dataset reader for Earth Engine data."""

def _get_base_key(key: Text) -> Text:
  """Extracts the base key from the provided key.

  Earth Engine exports TFRecords containing each data variable with its
  corresponding variable name. In the case of time sequences, the name of the
  data variable is of the form 'variable_1', 'variable_2', ..., 'variable_n',
  where 'variable' is the name of the variable, and n the number of elements
  in the time sequence. Extracting the base key ensures that each step of the
  time sequence goes through the same normalization steps.
  The base key obeys the following naming pattern: '([a-zA-Z]+)'
  For instance, for an input key 'variable_1', this function returns 'variable'.
  For an input key 'variable', this function simply returns 'variable'.

  Args:
    key: Input key.

  Returns:
    The corresponding base key.

  Raises:
    ValueError when `key` does not match the expected pattern.
  """
  match = re.match(r'([a-zA-Z]+)', key)
  if match:
    return match.group(1)
  raise ValueError(
      'The provided key does not match the expected pattern: {}'.format(key))


def _clip_and_rescale(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  """Clips and rescales inputs with the stats corresponding to `key`.

  Args:
    inputs: Inputs to clip and rescale.
    key: Key describing the inputs.

  Returns:
    Clipped and rescaled input.

  Raises:
    ValueError if there are no data statistics available for `key`.
  """
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, _, _ = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  return tf.math.divide_no_nan((inputs - min_val), (max_val - min_val))


def _clip_and_normalize(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  """Clips and normalizes inputs with the stats corresponding to `key`.

  Args:
    inputs: Inputs to clip and normalize.
    key: Key describing the inputs.

  Returns:
    Clipped and normalized input.

  Raises:
    ValueError if there are no data statistics available for `key`.
  """
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, mean, std = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  inputs = inputs - mean
  return tf.math.divide_no_nan(inputs, std)

def _get_features_dict(
    sample_size: int,
    features: List[Text],
) -> Dict[Text, tf.io.FixedLenFeature]:
  """Creates a features dictionary for TensorFlow IO.

  Args:
    sample_size: Size of the input tiles (square).
    features: List of feature names.

  Returns:
    A features dictionary for TensorFlow IO.
  """
  sample_shape = [sample_size, sample_size]
  features = set(features)
  columns = [
      tf.io.FixedLenFeature(shape=sample_shape, dtype=tf.float32)
      for _ in features
  ]
  return dict(zip(features, columns))


def _parse_fn(
    example_proto: tf.train.Example, data_size: int, sample_size: int,
    num_in_channels: int, clip_and_normalize: bool,
    clip_and_rescale: bool, random_crop: bool, center_crop: bool,
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Reads a serialized example.

  Args:
    example_proto: A TensorFlow example protobuf.
    data_size: Size of tiles (square) as read from input files.
    sample_size: Size the tiles (square) when input into the model.
    num_in_channels: Number of input channels.
    clip_and_normalize: True if the data should be clipped and normalized.
    clip_and_rescale: True if the data should be clipped and rescaled.
    random_crop: True if the data should be randomly cropped.
    center_crop: True if the data should be cropped in the center.

  Returns:
    (input_img, output_img) tuple of inputs and outputs to the ML model.
  """
  if (random_crop and center_crop):
    raise ValueError('Cannot have both random_crop and center_crop be True')
  input_features, output_features = INPUT_FEATURES, OUTPUT_FEATURES
  feature_names = input_features + output_features
  features_dict = _get_features_dict(data_size, feature_names)
  features = tf.io.parse_single_example(example_proto, features_dict)

  if clip_and_normalize:
    inputs_list = [
        _clip_and_normalize(features.get(key), key) for key in input_features
    ]
  elif clip_and_rescale:
    inputs_list = [
        _clip_and_rescale(features.get(key), key) for key in input_features
    ]
  else:
    inputs_list = [features.get(key) for key in input_features]
  
  inputs_stacked = tf.stack(inputs_list, axis=0)
  input_img = tf.transpose(inputs_stacked, [1, 2, 0])

  outputs_list = [features.get(key) for key in output_features]
  assert outputs_list, 'outputs_list should not be empty'
  outputs_stacked = tf.stack(outputs_list, axis=0)

  outputs_stacked_shape = outputs_stacked.get_shape().as_list()
  assert len(outputs_stacked.shape) == 3, ('outputs_stacked should be rank 3'
                                            'but dimensions of outputs_stacked'
                                            f' are {outputs_stacked_shape}')
  output_img = tf.transpose(outputs_stacked, [1, 2, 0])

  if random_crop:
    input_img, output_img = random_crop_input_and_output_images(
        input_img, output_img, sample_size, num_in_channels, 1)
  if center_crop:
    input_img, output_img = center_crop_input_and_output_images(
        input_img, output_img, sample_size)
  return input_img, output_img


def get_dataset(file_pattern: Text, data_size: int, sample_size: int,
                batch_size: int, num_in_channels: int, compression_type: Text,
                clip_and_normalize: bool, clip_and_rescale: bool,
                random_crop: bool, center_crop: bool) -> tf.data.Dataset:
  """Gets the dataset from the file pattern.

  Args:
    file_pattern: Input file pattern.
    data_size: Size of tiles (square) as read from input files.
    sample_size: Size the tiles (square) when input into the model.
    batch_size: Batch size.
    num_in_channels: Number of input channels.
    compression_type: Type of compression used for the input files.
    clip_and_normalize: True if the data should be clipped and normalized, False
      otherwise.
    clip_and_rescale: True if the data should be clipped and rescaled, False
      otherwise.
    random_crop: True if the data should be randomly cropped.
    center_crop: True if the data shoulde be cropped in the center.

  Returns:
    A TensorFlow dataset loaded from the input file pattern, with features
    described in the constants, and with the shapes determined from the input
    parameters to this function.
  """
  if (clip_and_normalize and clip_and_rescale):
    raise ValueError('Cannot have both normalize and rescale.')
  dataset = tf.data.Dataset.list_files(file_pattern)
  dataset = dataset.interleave(
      lambda x: tf.data.TFRecordDataset(x, compression_type=compression_type),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(
      lambda x: _parse_fn(  # pylint: disable=g-long-lambda
          x, data_size, sample_size, num_in_channels, clip_and_normalize,
          clip_and_rescale, random_crop, center_crop),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

Load the dataset.

The data are stored as 64x64 km regions. For each data sample, we extract a random 32x32 km region. In the following function call, we do not clip, rescale or normalize the data. 

In [9]:
side_length = 32 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
num_obs = 100 #batch size

dataset = get_dataset(
      file_pattern,
      data_size=64,
      sample_size=side_length,
      batch_size=num_obs,
      num_in_channels=12,
      compression_type=None,
      clip_and_normalize=False,
      clip_and_rescale=False,
      random_crop=True,
      center_crop=False)


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

TF Datasets are loaded lazily, so materialize the first batch of inputs and labels.

In [10]:
inputs, labels = next(iter(dataset)) 
#inputs_test, labels_test = next(iter(dataset_test)) 
#Are there two assignments happening on every iteration because dataset stores inputs with labels?
#print(inputs.shape) #(100, 32, 32, 12)
#print(labels.shape) #(100, 32, 32, 1)
#print(inputs[0, :, :, 11]) #Trying to grab the previous fire mask. (Apparent) success!
#print(labels[0,:, :, 0]) #Ok, I think the labels are the fire mask. (That also accords with standard usage of the term.)

I'd like to consider the average neighbor fire scores eventually. The function below helps me do that. Form: array_out = avg_neighbors(array_in)

In [11]:
#Eventually would like a function that takes in an input array of dimensions nxn, 
#outputs an array that gives avg of each cell's neighbors:
def avg_neighbors(array_in):
    #Check input
    if array_in.shape[0] != array_in.shape[1]:
        raise Exception('Only square arrays make sense here, since you\'re analyzing square arrays.')
    #Maybe should also do type-checking, but leave it for now.
    
    #Prepare the output array:
    n = array_in.shape[0] 
    array_out = np.zeros((n,n))
    
    #Guess who doesn't know how to do signal processing in Python...
    
    for i in range(n):
        for j in range(n):
            if i == 0:
                #Upper edge
                if j == 0:
                    #Upper left corner
                    sum_neighbors = array_in[i+1, j] + array_in[i, j+1] + array_in[i+1, j+1]
                    avg = sum_neighbors/3
                    
                elif j == (n-1):
                    #Upper right corner
                    sum_neighbors = array_in[i, j-1] + array_in[i+1, j] + array_in[i+1, j-1]
                    avg = sum_neighbors/3
                    
                else:
                    #Upper edge except corners
                    sum_neighbors = array_in[i, j-1] + array_in[i+1, j] + array_in[i, j+1] + array_in[i+1, j-1] + array_in[i+1, j+1]
                    avg = sum_neighbors/5
                   
            elif i == (n-1):
                #Lower edge
                if j == 0:
                    #Lower left corner
                    sum_neighbors = array_in[i-1, j] + array_in[i, j+1] + array_in[i-1, j+1]
                    avg = sum_neighbors/3
                    
                elif j == (n-1):
                    #Lower right corner
                    sum_neighbors = array_in[i, j-1] + array_in[i-1, j] + array_in[i-1, j-1]
                    avg = sum_neighbors/3
                    
                else:
                    #Lower edge except corners
                    sum_neighbors = array_in[i, j-1] + array_in[i, j+1] + array_in[i-1, j] + array_in[i-1, j-1] + array_in[i-1, j+1]
                    avg = sum_neighbors/5
                    
            else:
                if j == 0:
                    #Left edge except corners
                    sum_neighbors = array_in[i-1, j] + array_in[i+1, j] + array_in[i, j+1] + array_in[i-1, j+1] + array_in[i+1, j+1]
                    avg = sum_neighbors/5
                    
                elif j == (n-1):
                    #Right edge except corners
                    sum_neighbors = array_in[i-1, j] + array_in[i+1, j] + array_in[i, j-1] + array_in[i-1, j-1] + array_in[i+1, j-1]
                    avg = sum_neighbors/5
                    
                else:
                    #Not on any edge or corner
                    sum_neighbors = array_in[i, j+1] + array_in[i, j-1] + array_in[i-1, j] + array_in[i+1, j] + \
                                    array_in[i-1, j-1] + array_in[i-1, j+1] + array_in[i+1, j-1] + array_in[i+1, j+1]
                    avg = sum_neighbors/8
                    
                    
            array_out[i,j] = avg
            #/for loop body
        
    return array_out
                    
                    
    

Here, I test the function: 

In [12]:
#Test the function from above:
import numpy as np
arr = [[1,2,3], 
       [4,5,6],
       [7,8,9]]
arr = np.array(arr)
arr_avgs = avg_neighbors(arr)
print(arr_avgs)
#Good: appears to work

[[3.66666667 3.8        4.33333333]
 [4.6        5.         5.4       ]
 [5.66666667 6.2        6.33333333]]


In [13]:
#Let's experiment to see if I know what's going on in the code cell 35 above:
ls = ['Staring', 'at', 'the', 'tiny', 'planet', 'God', 'calculated', 'again']
ls_iter = iter(ls)
for i in range (len(ls)):
    print(next(ls_iter))
    
#Ok, so far so good
ls2 = [['There', 'was'], ['no', 'room'], ['for', 'a'], ['continuous', 'forest']]
ls2_iter = iter(ls2)
for i in range (len(ls2)):
    first_word, second_word = next(ls2_iter)
    print(first_word, second_word)
#Yes, ok, working as expected




Staring
at
the
tiny
planet
God
calculated
again
There was
no room
for a
continuous forest


In [14]:
#Let's try using the function above on the previous fire masks:
#This code (is supposed to) find the first "interesting" previous fire mask and compute the avg number of neighboring 
#on-fire cells for each cell

prev_fire_masks = inputs[:, :, :, 11] #observation number, pixel row, pixel col
#prev_fire_masks_test = inputs_test[:, :, :, 11] #observation number, pixel row, pixel col

found_it_flag = 0 #will set to 1 once we've found our "interesting" fire mask
img_num = 0
while found_it_flag == 0:
    fire_mask = np.array(prev_fire_masks[img_num, :, :])
    if (np.all( (fire_mask == 0)) ): #if boring picture where there's no fire, toss it
        img_num = img_num + 1
    elif (np.all( np.invert(fire_mask == -1) )): #if NO data is missing data, cond. is TRUE --> you want this one
        test_img = fire_mask
        found_it_flag = 1
    else:
        img_num = img_num + 1

np.set_printoptions(threshold=np.inf) #this just stops Jupyter from truncating the output
print('fire mask:\n', fire_mask, '\n\n')
print('computed avg neighbor fire mask:\n', avg_neighbors(fire_mask))
#Note: don't freak out when you see the second matrix "look bigger" than the first. The dimensions are correct; the second matrix
#is just visually larger because instead of 0s and 1s its entries are 3fs.
#Tested with small grids (9x9) and appears to work.

fire mask:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 

In the next several cells, we eliminate observations where there is missing or uncertain data in the previous fire mask.

In [15]:
#Try to eliminate all observations where there are uncertain squares in the previous fire mask.
prev_masks_array = np.array(inputs[:, :, :, 11])
#print(prev_masks_array.shape) #100x32x32—good!

#Build the array of certain data AND SAVE THE INDICES
first_find_flag = 1
count = 0
indices = []

for img_num in range(num_obs): 
    fire_mask = np.array(prev_fire_masks[img_num, :, :])
    if (np.all( np.invert(fire_mask == -1) )): #If no missing data, condition is TRUE.
        count += 1
        indices.append(img_num)
        if first_find_flag == 1: #If you need to start the array
            certain_prev_fire_masks = fire_mask
            first_find_flag = 0  #Remember to turn the flag off!
        else:
            certain_prev_fire_masks = np.dstack((certain_prev_fire_masks, fire_mask)) #d
            

#Test: You want the printouts from the following three lines to be consistent
print(certain_prev_fire_masks.shape)
print(count) 
print(len(indices))
#Good, they are.
        

(32, 32, 75)
75
75


In [16]:
#Now that I've found the observations with "pristine" previous fire masks, I'd like to get those observations' features matrices
full_input_array = np.array(inputs) #100x32x32x12

for i, index in enumerate(indices):
    if i == 0:
        certain_input_array = full_input_array[index,:,:,:]
        print(certain_input_array.shape) #32x32x12
    elif i == 1:
        certain_input_array = np.concatenate((certain_input_array[..., np.newaxis], full_input_array[index,:,:,:, np.newaxis]), axis=3)
    else:
        certain_input_array = np.concatenate((certain_input_array, full_input_array[index,:,:,:, np.newaxis]), axis=3)
        
print(certain_input_array.shape)
#SUCCESS! :) :) :) 
#certain_input_array now holds only the 77 observations with certain previous fire masks


(32, 32, 12)
(32, 32, 12, 75)


In [17]:
#Also want the labels at only these indices and the average neighbor value matrices
full_labels = np.array(labels)
#print(labels.shape) 100x32x32x1, as expected

for i, index in enumerate(indices):
    if i == 0:
        #labels
        certain_labels = full_labels[index,:,:,:] #32x32x1
        
        #avg neighbor values
        surrounding_fire_scores = avg_neighbors(full_labels[index,:,:,:])
        surrounding_fire_scores = surrounding_fire_scores[..., np.newaxis]
        
    else:
        #labels
        certain_labels = np.concatenate((certain_labels, full_labels[index,:,:,:]), axis=2)
        
        #avg neighbor values
        avg_mat = avg_neighbors(full_labels[index,:,:,:])
        surrounding_fire_scores = np.concatenate((surrounding_fire_scores, avg_mat[...,np.newaxis]), axis=2)

#Printouts from following lines should agree...
print(certain_labels.shape) #32x32x77 
print(surrounding_fire_scores.shape) #32x32x77 
#... and they do! :)

(32, 32, 75)
(32, 32, 75)


In [18]:
#Function to identify certain observations in the previous fire mask and return:
#1) a list of their indices in the batch (need this to grab the right ones from the labels) and 
#2) the actual array of certain observations

def elim_uncertain(prev_fire_mask_batch):
    
    prev_masks_array = np.array(prev_fire_mask_batch)
    print(prev_masks_array.shape)
    num_imgs, rows, cols = prev_masks_array.shape
    #Build the array of certain data AND SAVE THE INDICES
    first_find_flag = 1
    count = 0
    indices = []

    for img_num in range(num_imgs): 
        fire_mask = prev_fire_mask_batch[img_num, :, :] #grab the "working fire mask" off the pile
        
        if (np.all( np.invert(fire_mask == -1) )): #If no missing data, condition is TRUE.
            count += 1
            indices.append(img_num)
            if first_find_flag == 1: #If you need to start the array
                certain_prev_fire_masks_batch = fire_mask
                first_find_flag = 0  #Remember to turn the flag off!
            else:
                certain_prev_fire_masks_batch = np.dstack((certain_prev_fire_masks_batch, fire_mask)) 
    
    return certain_prev_fire_masks_batch, indices

In [19]:
#Function to extract only the labels (i.e. current fire masks) from the certain observations
def extract_certain_labels(certain_indices, og_labels):
    
    for i, index in enumerate(certain_indices):
        if i == 0:
            extracted_labels = og_labels[index,:,:,:] #the og_labels dimensions are batch_size by sidelength by sidelength by 1
        else:
            #labels
            extracted_labels = np.concatenate((extracted_labels, og_labels[index,:,:,:]), axis=2)

    return extracted_labels

In [20]:
#Create multi-D array of neighbor fire values
def avg_neighbor_batch(batch_in):
    rows, cols, batch_size = batch_in.shape #ordering of dimensions here meant to be compatible with elim_uncertain and extract_certain_labels
    batch_out = np.zeros((rows, cols, batch_size))
    for i in range(batch_size):
        working_arr = batch_in[:,:,i]
        avgd_arr = avg_neighbors(working_arr)
        batch_out[:,:,i] =  avgd_arr
    #/for loop
    
    return batch_out

In [21]:
#Create multi-D array of distance neighbor fire values
def distance_batch(batch_in):
    rows, cols, batch_size = batch_in.shape #ordering of dimensions here meant to be compatible with elim_uncertain and extract_certain_labels
    batch_out = np.zeros((rows, cols, batch_size))
    for i in range(batch_size):
        working_arr = batch_in[:,:,i]
        calculated_arr = calc_distance(working_arr)
        batch_out[:,:,i] =  calculated_arr
    #/for loop
    
    return batch_out

In [22]:
#eliminate the indices who were unceartin in the other features too
def elim_indices(indices_to_keep, feature_mask):
    #mask_array = np.array(feature_mask)
    feature_mask_elim = []

    for ind in indices_to_keep: 
        feature_mask_elim.append(feature_mask[ind, :, :])
    
    feature_mask_elim = np.array(feature_mask_elim)
    #print(len(feature_mask_elim))
    return feature_mask_elim

In [23]:
import math
def calc_distance(matrix):
    rows, cols = len(matrix), len(matrix[0])
    max_distance = math.sqrt(rows**2 + cols**2) 
    output = [[max_distance for _ in range(cols)] for _ in range(rows)]
        
    for i in range(rows):
        for j in range(cols):
            if matrix[i][j] == 1:
                output[i][j] = 0
            else:
                for x in range(rows):
                    for y in range(cols):
                        if matrix[x][y] == 1:
                            euclidean_distance = math.sqrt((i - x)**2 + (j - y)**2)
                            distance = 1 - 1/euclidean_distance if euclidean_distance != 0 else 0
                            output[i][j] = min(output[i][j], distance)
    
    # Format the output to remove the decimal from zeros
    formatted_output = [[int(value) if value == 0 else value for value in row] for row in output]
    return formatted_output



In [24]:
#test calc_distance function
array_10x10 = [
    [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
]

distance_matrix = calc_distance(array_10x10)
original = np.array(array_10x10)
print(original, "\n")
print(np.array(distance_matrix))

[[0 0 0 1 1 0 0 0 1 0]
 [1 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 1 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]] 

[[0.         0.29289322 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.29289322 0.         0.         0.
  0.         0.         0.         0.29289322]
 [0.         0.         0.         0.29289322 0.         0.29289322
  0.         0.         0.         0.5       ]
 [0.29289322 0.         0.29289322 0.         0.         0.
  0.         0.         0.29289322 0.5527864 ]
 [0.5527864  0.5        0.29289322 0.         0.         0.
  0.         0.29289322 0.         0.29289322]
 [0.29289322 0.         0.         0.         0.         0.29289322
  0.5        0.         0.         0.        ]
 [0.         0.         0.         0.         0.29289322 0.29289322
  0.         0.         0.         0.    

In [26]:
#Test functions above:
import numpy as np
full_prev_fire_masks = np.array(inputs[:,:,:,11]) #just grabs the previous fire masks from the full observation multi-D array
full_curr_fire_masks = np.array(labels) #100x32x32x1

certain_prev_masks, certain_indices = elim_uncertain(full_prev_fire_masks) #32x32x86 (generally: sidelength^2 x #certain obs.)
certain_labels = extract_certain_labels(certain_indices, full_curr_fire_masks) #32x32x86 (generally: sidelength^2 x #certain obs.)
avg_neighbors_feat = avg_neighbor_batch(certain_prev_masks) #32x32x86 (generally: sidelength^2 x #certain obs.)
euclid_distance = distance_batch(certain_prev_masks)
print(len(avg_neighbors_feat))
print(len(euclid_distance))

#Imple neu
cleaned_elevation = elim_indices(certain_indices, np.array(inputs[:,:,:,0]))
cleaned_wind_dir = elim_indices(certain_indices, np.array(inputs[:,:,:,1]))
cleaned_wind_velo = elim_indices(certain_indices, np.array(inputs[:,:,:,2]))
cleaned_temp_min = elim_indices(certain_indices, np.array(inputs[:,:,:,3]))
cleaned_temp_max = elim_indices(certain_indices, np.array(inputs[:,:,:,4]))
cleaned_humidity = elim_indices(certain_indices, np.array(inputs[:,:,:,5]))
cleaned_precipitation = elim_indices(certain_indices, np.array(inputs[:,:,:,6]))
cleaned_drought = elim_indices(certain_indices, np.array(inputs[:,:,:,7]))
cleaned_vegetation = elim_indices(certain_indices, np.array(inputs[:,:,:,8]))
cleaned_population = elim_indices(certain_indices, np.array(inputs[:,:,:,9]))
cleaned_ERC = elim_indices(certain_indices, np.array(inputs[:,:,:,10]))
#print(cleaned_elevation)
#Success! Note that new format for certain_labels and certain_prev_masks is rows, cols, index in (new, certain-only) pile.

(100, 32, 32)
32
32


# Decision Tree

In [48]:
#Benchmark method: Try linear regression based on only:
#1) whether a given pixel is currently on fire and 
#2) number of neighbors that are currently on fire

#Vectorize previous fire masks
flat_prev_masks = []

#Remembering that Python is zero-indexed is going to be VERY IMPORTANT in the below!
for obs in range(certain_prev_masks.shape[2]):
    for row in range(certain_prev_masks.shape[0]):
        for col in range(certain_prev_masks.shape[1]):
            flat_prev_masks.append(certain_prev_masks[row, col, obs])
flat_prev_masks = np.array(flat_prev_masks)

#Vectorize euc distance
flat_euclid_distance = []
for obs in range(euclid_distance.shape[2]):
    for row in range(euclid_distance.shape[0]):
        for col in range(euclid_distance.shape[1]):
            flat_euclid_distance.append(euclid_distance[row, col, obs])
flat_euclid_distance = np.array(flat_euclid_distance)

flat_elevation = []
for obs in range(cleaned_elevation.shape[2]):
    for row in range(cleaned_elevation.shape[0]):
        for col in range(cleaned_elevation.shape[1]):
            flat_elevation.append(cleaned_elevation[row, col, obs])
flat_elevation = np.array(flat_elevation)

flat_wind_dir = []
for obs in range(cleaned_wind_dir.shape[2]):
    for row in range(cleaned_wind_dir.shape[0]):
        for col in range(cleaned_wind_dir.shape[1]):
            flat_wind_dir.append(cleaned_wind_dir[row, col, obs])
flat_wind_dir = np.array(flat_wind_dir)

flat_wind_velo = []
for obs in range(cleaned_wind_velo.shape[2]):
    for row in range(cleaned_wind_velo.shape[0]):
        for col in range(cleaned_wind_velo.shape[1]):
            flat_wind_velo.append(cleaned_wind_velo[row, col, obs])
flat_wind_velo = np.array(flat_wind_velo)

flat_temp_min = []
for obs in range(cleaned_temp_min.shape[2]):
    for row in range(cleaned_temp_min.shape[0]):
        for col in range(cleaned_temp_min.shape[1]):
            flat_temp_min.append(cleaned_temp_min[row, col, obs])
flat_temp_min = np.array(flat_temp_min)

flat_temp_max = []
for obs in range(cleaned_temp_max.shape[2]):
    for row in range(cleaned_temp_max.shape[0]):
        for col in range(cleaned_temp_max.shape[1]):
            flat_temp_max.append(cleaned_temp_max[row, col, obs])
flat_temp_max = np.array(flat_temp_max)

flat_humidity = []
for obs in range(cleaned_humidity.shape[2]):
    for row in range(cleaned_humidity.shape[0]):
        for col in range(cleaned_humidity.shape[1]):
            flat_humidity.append(cleaned_humidity[row, col, obs])
flat_humidity = np.array(flat_humidity)

flat_precipitation = []
for obs in range(cleaned_precipitation.shape[2]):
    for row in range(cleaned_precipitation.shape[0]):
        for col in range(cleaned_precipitation.shape[1]):
            flat_precipitation.append(cleaned_precipitation[row, col, obs])
flat_precipitation = np.array(flat_precipitation)

flat_drought = []
for obs in range(cleaned_drought.shape[2]):
    for row in range(cleaned_drought.shape[0]):
        for col in range(cleaned_drought.shape[1]):
            flat_drought.append(cleaned_drought[row, col, obs])
flat_drought = np.array(flat_drought)

flat_vegetation = []
for obs in range(cleaned_vegetation.shape[2]):
    for row in range(cleaned_vegetation.shape[0]):
        for col in range(cleaned_vegetation.shape[1]):
            flat_vegetation.append(cleaned_vegetation[row, col, obs])
flat_vegetation = np.array(flat_vegetation)

flat_population = []
for obs in range(cleaned_population.shape[2]):
    for row in range(cleaned_population.shape[0]):
        for col in range(cleaned_population.shape[1]):
            flat_population.append(cleaned_population[row, col, obs])
flat_population = np.array(flat_population)

flat_ERC = []
for obs in range(cleaned_ERC.shape[2]):
    for row in range(cleaned_ERC.shape[0]):
        for col in range(cleaned_ERC.shape[1]):
            flat_ERC.append(cleaned_ERC[row, col, obs])
flat_ERC = np.array(flat_ERC)

#print(flat_prev_masks.shape)
#print(flat_avg_nbrs.shape)
X_TRAIN_FEATURES = ['prev_fire', 
                    'euclid_distance', 
                    'elevation', 'wind_dir', 'wind_velo', 'temp_min', 'temp_max', 'humidity', 'precipitation', 'drought', 'vegetation', 'population', 'ERC']
X_train = np.vstack((
    np.transpose(flat_prev_masks), 
    np.transpose(flat_euclid_distance),
    np.transpose(flat_elevation),
    np.transpose(flat_wind_dir),
    np.transpose(flat_wind_velo),
    np.transpose(flat_temp_min),
    np.transpose(flat_temp_max),
    np.transpose(flat_humidity),
    np.transpose(flat_precipitation),
    np.transpose(flat_drought),
    np.transpose(flat_vegetation),
    np.transpose(flat_population),
    np.transpose(flat_ERC)
))

X_train = np.transpose(X_train) #so that observations correspond to rows now

#Vectorize labels
flat_labels = []
for obs in range(certain_labels.shape[2]):
    for row in range(certain_labels.shape[0]):
        for col in range(certain_labels.shape[1]):
            flat_labels.append(certain_labels[row, col, obs])
flat_labels = np.array(flat_labels)

Y_train = flat_labels
#print(Y_train.shape)

variables = ["currently on fire?", "# of neighbors currently on fire"]



from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score

X_train_r, X_test_r, Y_train_r, Y_test_r = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

model = DecisionTreeClassifier(criterion = 'gini', max_depth = 3, min_samples_leaf = 3)
#model.fit(X_train, Y_train)


#print('training R2-score:', np.round(   r2_score(  Y_train, lr_fire.predict(X_train)),2  )   ) 

k = 10  # for example, you can change this value as needed
scores = cross_val_score(model, X_train, Y_train, cv=k)
#print("Cross Val: ", scores.mean())
#model = RandomForestClassifier(criterion = "gini", n_estimators = 15)
model.fit(X_train_r, Y_train_r)

tree = export_graphviz(model, None,
                      feature_names = ["prev Fire", "euc_distance", " elevation", "wind_dir",
                                      "wind_velo", "temp_min", "temp_max", "humidity", 
                                      "precipitation", "drought", "vegatation", "population", "ERC"],
                      rounded = True, filled = True)
#print("normal score: ",model.score(X_test_r, Y_test_r))
#graphviz.Source(tree)




importances = model.feature_importances_

# Get the indices of the features sorted by importance
indices = np.argsort(importances)[::-1]

# Get the feature names
feature_names = X_TRAIN_FEATURES
#print(X_train_r.shape[1], "\n")
# Display the feature importances
#print("Feature ranking:")

#for f in range(X_train_r.shape[1]):
#    print(f"{f + 1}. {feature_names[indices[f]]}: {importances[indices[f]]:.4f}")

#Plot the feature importances
#plt.figure(figsize=(10, 6))
#plt.title("Feature importances")
#plt.bar(range(X_train_r.shape[1]), importances[indices], align="center")
#plt.xticks(range(X_train_r.shape[1]), np.array(feature_names)[indices], rotation=45)
#plt.xlim([-1, X_train_r.shape[1]])
#plt.grid(axis='y', linestyle='--', linewidth=0.7, alpha=0.6)
#plt.show()

76800
76800


# Random Forest

In [29]:
#Benchmark method: Try linear regression based on only:
#1) whether a given pixel is currently on fire and 
#2) number of neighbors that are currently on fire

#Vectorize previous fire masks
flat_prev_masks = []

#Remembering that Python is zero-indexed is going to be VERY IMPORTANT in the below!
for obs in range(certain_prev_masks.shape[2]):
    for row in range(certain_prev_masks.shape[0]):
        for col in range(certain_prev_masks.shape[1]):
            flat_prev_masks.append(certain_prev_masks[row, col, obs])
flat_prev_masks = np.array(flat_prev_masks)

#Vectorize euc distance
flat_euclid_distance = []
for obs in range(euclid_distance.shape[2]):
    for row in range(euclid_distance.shape[0]):
        for col in range(euclid_distance.shape[1]):
            flat_euclid_distance.append(euclid_distance[row, col, obs])
flat_euclid_distance = np.array(flat_euclid_distance)

flat_elevation = []
for obs in range(cleaned_elevation.shape[2]):
    for row in range(cleaned_elevation.shape[0]):
        for col in range(cleaned_elevation.shape[1]):
            flat_elevation.append(cleaned_elevation[row, col, obs])
flat_elevation = np.array(flat_elevation)

flat_wind_dir = []
for obs in range(cleaned_wind_dir.shape[2]):
    for row in range(cleaned_wind_dir.shape[0]):
        for col in range(cleaned_wind_dir.shape[1]):
            flat_wind_dir.append(cleaned_wind_dir[row, col, obs])
flat_wind_dir = np.array(flat_wind_dir)

flat_wind_velo = []
for obs in range(cleaned_wind_velo.shape[2]):
    for row in range(cleaned_wind_velo.shape[0]):
        for col in range(cleaned_wind_velo.shape[1]):
            flat_wind_velo.append(cleaned_wind_velo[row, col, obs])
flat_wind_velo = np.array(flat_wind_velo)

flat_temp_min = []
for obs in range(cleaned_temp_min.shape[2]):
    for row in range(cleaned_temp_min.shape[0]):
        for col in range(cleaned_temp_min.shape[1]):
            flat_temp_min.append(cleaned_temp_min[row, col, obs])
flat_temp_min = np.array(flat_temp_min)

flat_temp_max = []
for obs in range(cleaned_temp_max.shape[2]):
    for row in range(cleaned_temp_max.shape[0]):
        for col in range(cleaned_temp_max.shape[1]):
            flat_temp_max.append(cleaned_temp_max[row, col, obs])
flat_temp_max = np.array(flat_temp_max)

flat_humidity = []
for obs in range(cleaned_humidity.shape[2]):
    for row in range(cleaned_humidity.shape[0]):
        for col in range(cleaned_humidity.shape[1]):
            flat_humidity.append(cleaned_humidity[row, col, obs])
flat_humidity = np.array(flat_humidity)

flat_precipitation = []
for obs in range(cleaned_precipitation.shape[2]):
    for row in range(cleaned_precipitation.shape[0]):
        for col in range(cleaned_precipitation.shape[1]):
            flat_precipitation.append(cleaned_precipitation[row, col, obs])
flat_precipitation = np.array(flat_precipitation)

flat_drought = []
for obs in range(cleaned_drought.shape[2]):
    for row in range(cleaned_drought.shape[0]):
        for col in range(cleaned_drought.shape[1]):
            flat_drought.append(cleaned_drought[row, col, obs])
flat_drought = np.array(flat_drought)

flat_vegetation = []
for obs in range(cleaned_vegetation.shape[2]):
    for row in range(cleaned_vegetation.shape[0]):
        for col in range(cleaned_vegetation.shape[1]):
            flat_vegetation.append(cleaned_vegetation[row, col, obs])
flat_vegetation = np.array(flat_vegetation)

flat_population = []
for obs in range(cleaned_population.shape[2]):
    for row in range(cleaned_population.shape[0]):
        for col in range(cleaned_population.shape[1]):
            flat_population.append(cleaned_population[row, col, obs])
flat_population = np.array(flat_population)

flat_ERC = []
for obs in range(cleaned_ERC.shape[2]):
    for row in range(cleaned_ERC.shape[0]):
        for col in range(cleaned_ERC.shape[1]):
            flat_ERC.append(cleaned_ERC[row, col, obs])
flat_ERC = np.array(flat_ERC)

#print(flat_prev_masks.shape)
#print(flat_avg_nbrs.shape)
X_TRAIN_FEATURES = ['prev_fire', 
                    'euclid_distance', 
                    'elevation', 'wind_dir', 'wind_velo', 'temp_min', 'temp_max', 'humidity', 'precipitation', 'drought', 'vegetation', 'population', 'ERC']
X_train = np.vstack((
    np.transpose(flat_prev_masks), 
    np.transpose(flat_euclid_distance),
    np.transpose(flat_elevation),
    np.transpose(flat_wind_dir),
    np.transpose(flat_wind_velo),
    np.transpose(flat_temp_min),
    np.transpose(flat_temp_max),
    np.transpose(flat_humidity),
    np.transpose(flat_precipitation),
    np.transpose(flat_drought),
    np.transpose(flat_vegetation),
    np.transpose(flat_population),
    np.transpose(flat_ERC)
))

X_train = np.transpose(X_train) #so that observations correspond to rows now

#Vectorize labels
flat_labels = []
for obs in range(certain_labels.shape[2]):
    for row in range(certain_labels.shape[0]):
        for col in range(certain_labels.shape[1]):
            flat_labels.append(certain_labels[row, col, obs])
flat_labels = np.array(flat_labels)

Y_train = flat_labels
#print(Y_train.shape)

variables = ["currently on fire?", "# of neighbors currently on fire"]



from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


X_train_r, X_test_r, Y_train_r, Y_test_r = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

model = RandomForestClassifier(criterion = "gini", n_estimators = 15)
model.fit(X_train_r, Y_train_r)
print(model.score(X_test_r, Y_test_r))


X = np.column_stack((X_train_r, model.predict_proba(X_train_r)[:, 1]))  # Combining original features with RF predictions
Y = Y_train_r  # Adjust according to your data

param = {
    'max_depth': 3,
    'eta': 0.3,
    'objective': 'reg:squarederror'
}
num_round = 50
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
accuracies = []

for train_index, val_index in kf.split(X):
    X_train_r, X_val = X[train_index], X[val_index]
    Y_train_r, Y_val = Y[train_index], Y[val_index]
    
    dtrain = xgb.DMatrix(X_train_r, label=Y_train_r)
    dval = xgb.DMatrix(X_val, label=Y_val)
    
    bst = xgb.train(param, dtrain, num_round)
    
    predictions = bst.predict(dval)
    predicted_labels = [1 if p > 0.5 else 0 for p in predictions]
    
    acc = accuracy_score(Y_val, predicted_labels)
    accuracies.append(acc)

average_accuracy = np.mean(accuracies)
print(f"Average Accuracy across folds: {average_accuracy:.4f}")

0.9796223958333333
Average Accuracy across folds: 0.9968


# Gridsearch

In [30]:
#Benchmark method: Try linear regression based on only:
#1) whether a given pixel is currently on fire and 
#2) number of neighbors that are currently on fire

#Vectorize previous fire masks
flat_prev_masks = []

#Remembering that Python is zero-indexed is going to be VERY IMPORTANT in the below!
for obs in range(certain_prev_masks.shape[2]):
    for row in range(certain_prev_masks.shape[0]):
        for col in range(certain_prev_masks.shape[1]):
            flat_prev_masks.append(certain_prev_masks[row, col, obs])
flat_prev_masks = np.array(flat_prev_masks)

#Vectorize euc distance
flat_euclid_distance = []
for obs in range(euclid_distance.shape[2]):
    for row in range(euclid_distance.shape[0]):
        for col in range(euclid_distance.shape[1]):
            flat_euclid_distance.append(euclid_distance[row, col, obs])
flat_euclid_distance = np.array(flat_euclid_distance)

flat_elevation = []
for obs in range(cleaned_elevation.shape[2]):
    for row in range(cleaned_elevation.shape[0]):
        for col in range(cleaned_elevation.shape[1]):
            flat_elevation.append(cleaned_elevation[row, col, obs])
flat_elevation = np.array(flat_elevation)

flat_wind_dir = []
for obs in range(cleaned_wind_dir.shape[2]):
    for row in range(cleaned_wind_dir.shape[0]):
        for col in range(cleaned_wind_dir.shape[1]):
            flat_wind_dir.append(cleaned_wind_dir[row, col, obs])
flat_wind_dir = np.array(flat_wind_dir)

flat_wind_velo = []
for obs in range(cleaned_wind_velo.shape[2]):
    for row in range(cleaned_wind_velo.shape[0]):
        for col in range(cleaned_wind_velo.shape[1]):
            flat_wind_velo.append(cleaned_wind_velo[row, col, obs])
flat_wind_velo = np.array(flat_wind_velo)

flat_temp_min = []
for obs in range(cleaned_temp_min.shape[2]):
    for row in range(cleaned_temp_min.shape[0]):
        for col in range(cleaned_temp_min.shape[1]):
            flat_temp_min.append(cleaned_temp_min[row, col, obs])
flat_temp_min = np.array(flat_temp_min)

flat_temp_max = []
for obs in range(cleaned_temp_max.shape[2]):
    for row in range(cleaned_temp_max.shape[0]):
        for col in range(cleaned_temp_max.shape[1]):
            flat_temp_max.append(cleaned_temp_max[row, col, obs])
flat_temp_max = np.array(flat_temp_max)

flat_humidity = []
for obs in range(cleaned_humidity.shape[2]):
    for row in range(cleaned_humidity.shape[0]):
        for col in range(cleaned_humidity.shape[1]):
            flat_humidity.append(cleaned_humidity[row, col, obs])
flat_humidity = np.array(flat_humidity)

flat_precipitation = []
for obs in range(cleaned_precipitation.shape[2]):
    for row in range(cleaned_precipitation.shape[0]):
        for col in range(cleaned_precipitation.shape[1]):
            flat_precipitation.append(cleaned_precipitation[row, col, obs])
flat_precipitation = np.array(flat_precipitation)

flat_drought = []
for obs in range(cleaned_drought.shape[2]):
    for row in range(cleaned_drought.shape[0]):
        for col in range(cleaned_drought.shape[1]):
            flat_drought.append(cleaned_drought[row, col, obs])
flat_drought = np.array(flat_drought)

flat_vegetation = []
for obs in range(cleaned_vegetation.shape[2]):
    for row in range(cleaned_vegetation.shape[0]):
        for col in range(cleaned_vegetation.shape[1]):
            flat_vegetation.append(cleaned_vegetation[row, col, obs])
flat_vegetation = np.array(flat_vegetation)

flat_population = []
for obs in range(cleaned_population.shape[2]):
    for row in range(cleaned_population.shape[0]):
        for col in range(cleaned_population.shape[1]):
            flat_population.append(cleaned_population[row, col, obs])
flat_population = np.array(flat_population)

flat_ERC = []
for obs in range(cleaned_ERC.shape[2]):
    for row in range(cleaned_ERC.shape[0]):
        for col in range(cleaned_ERC.shape[1]):
            flat_ERC.append(cleaned_ERC[row, col, obs])
flat_ERC = np.array(flat_ERC)

#print(flat_prev_masks.shape)
#print(flat_avg_nbrs.shape)
X_TRAIN_FEATURES = ['prev_fire', 
                    'euclid_distance', 
                    'elevation', 'wind_dir', 'wind_velo', 'temp_min', 'temp_max', 'humidity', 'precipitation', 'drought', 'vegetation', 'population', 'ERC']
X_train = np.vstack((
    np.transpose(flat_prev_masks), 
    np.transpose(flat_euclid_distance),
    np.transpose(flat_elevation),
    np.transpose(flat_wind_dir),
    np.transpose(flat_wind_velo),
    np.transpose(flat_temp_min),
    np.transpose(flat_temp_max),
    np.transpose(flat_humidity),
    np.transpose(flat_precipitation),
    np.transpose(flat_drought),
    np.transpose(flat_vegetation),
    np.transpose(flat_population),
    np.transpose(flat_ERC)
))

X_train = np.transpose(X_train) #so that observations correspond to rows now

#Vectorize labels
flat_labels = []
for obs in range(certain_labels.shape[2]):
    for row in range(certain_labels.shape[0]):
        for col in range(certain_labels.shape[1]):
            flat_labels.append(certain_labels[row, col, obs])
flat_labels = np.array(flat_labels)

Y_train = flat_labels
#print(Y_train.shape)

variables = ["currently on fire?", "# of neighbors currently on fire"]



from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


X_train_r, X_test_r, Y_train_r, Y_test_r = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

model = RandomForestClassifier(criterion = "gini", n_estimators = 15)
model.fit(X_train_r, Y_train_r)
print(model.score(X_test_r, Y_test_r))

X = np.column_stack((X_train_r, model.predict_proba(X_train_r)[:, 1]))  # Combining original features with RF predictions
Y = Y_train_r  # Adjust according to your data

# GridSearchCV implementation
param_grid = {
    'max_depth': [3, 4, 5],
    'eta': [0.1, 0.2, 0.3],
    'objective': ['reg:squarederror'],
    'n_estimators': [10, 50, 100]
}

# Convert the data to DMatrix format
dtrain = xgb.DMatrix(X, label=Y)

# Create a train/test split for the grid search validation
X_train_gs, X_val_gs, Y_train_gs, Y_val_gs = train_test_split(X, Y, test_size=0.20, random_state=42)

# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Set up the grid search
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit the model
grid_search.fit(X_train_gs, Y_train_gs)

# Print the best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Predict on the validation set
predicted_labels_gs = grid_search.predict(X_val_gs)
acc_gs = accuracy_score(Y_val_gs, predicted_labels_gs)
print(f"Accuracy on validation set: {acc_gs:.4f}")

# Continue with your KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
accuracies = []

for train_index, val_index in kf.split(X):
    X_train_r, X_val = X[train_index], X[val_index]
    Y_train_r, Y_val = Y[train_index], Y[val_index]
    
    dtrain = xgb.DMatrix(X_train_r, label=Y_train_r)
    dval = xgb.DMatrix(X_val, label=Y_val)
    
    bst = xgb.train(grid_search.best_params_, dtrain, grid_search.best_params_['n_estimators'])
    
    predictions = bst.predict(dval)
    predicted_labels = [1 if p > 0.5 else 0 for p in predictions]
    
    acc = accuracy_score(Y_val, predicted_labels)
    accuracies.append(acc)

average_accuracy = np.mean(accuracies)
print(f"Average Accuracy across folds using best parameters: {average_accuracy:.4f}")

0.9805338541666667
Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:48:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:48:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:48:41] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:48:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the ol

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  9.0min finished


[00:57:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best parameters found:  {'eta': 0.2, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror'}
Best cross-validation score: 1.00
Accuracy on validation set: 0.9986
[00:57:53] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[00:57:59] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly

## Autoencoder

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.Encoder = nn.Sequential(
            nn.Conv2d(3,64,kernel_size = (8,8),padding = 2,padding_mode = 'replicate',bias = True),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size = (2,2)),
            ###Now 128*128*64
            nn.Conv2d(64,64,kernel_size = (5,5),padding = 1,padding_mode = 'replicate',bias = True),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size = (2,2)),
            ###Now 64*64*32
            nn.Conv2d(64,32,kernel_size = (3,3),padding = 2,padding_mode = 'replicate',bias = True),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size = (2,2)),
            ###Now 32*32*16
            #nn.Conv2d(16,16,kernel_size = (10,10),padding_mode = 'same',bias = True),
            #nn.LeakyReLU(0.1),
            #nn.MaxPool2d(kernel_size = (2,2)))
        )
            ###Now 16*16*16

        self.Decoder = nn.Sequential(
            #nn.ConvTranspose2d(16,16,kernel_size = (3,3)),
            #nn.LeakyReLU(0.1),
            nn.ConvTranspose2d(32,64,kernel_size = (3,3),stride = 2, padding = 1),
            nn.LeakyReLU(0.1),
            nn.ConvTranspose2d(64,64,kernel_size = (6,6),stride = 2,padding = 1),
            nn.LeakyReLU(0.1),
            nn.ConvTranspose2d(64,3,kernel_size = (8,8), stride = 2, padding = 3),
            nn.Tanh())

    def forward(self,x,corruption):
        x = self.Encoder(x)
            #print('Encoder output: ',x.shape)
        if x.shape == corruption.shape:
            x = x * corruption
        else:
            corruption = corruption[0:len(x)]
            x = x * corruption
        x = self.Decoder(x)
        #print('Decoder output: ',x.shape)
        return x

In [39]:
#Vectorize previous fire masks
flat_prev_masks = []

#Remembering that Python is zero-indexed is going to be VERY IMPORTANT in the below!
for obs in range(certain_prev_masks.shape[2]):
    for row in range(certain_prev_masks.shape[0]):
        for col in range(certain_prev_masks.shape[1]):
            flat_prev_masks.append(certain_prev_masks[row, col, obs])
flat_prev_masks = np.array(flat_prev_masks)

#Vectorize euc distance
flat_euclid_distance = []
for obs in range(euclid_distance.shape[2]):
    for row in range(euclid_distance.shape[0]):
        for col in range(euclid_distance.shape[1]):
            flat_euclid_distance.append(euclid_distance[row, col, obs])
flat_euclid_distance = np.array(flat_euclid_distance)

flat_elevation = []
for obs in range(cleaned_elevation.shape[2]):
    for row in range(cleaned_elevation.shape[0]):
        for col in range(cleaned_elevation.shape[1]):
            flat_elevation.append(cleaned_elevation[row, col, obs])
flat_elevation = np.array(flat_elevation)

flat_wind_dir = []
for obs in range(cleaned_wind_dir.shape[2]):
    for row in range(cleaned_wind_dir.shape[0]):
        for col in range(cleaned_wind_dir.shape[1]):
            flat_wind_dir.append(cleaned_wind_dir[row, col, obs])
flat_wind_dir = np.array(flat_wind_dir)

flat_wind_velo = []
for obs in range(cleaned_wind_velo.shape[2]):
    for row in range(cleaned_wind_velo.shape[0]):
        for col in range(cleaned_wind_velo.shape[1]):
            flat_wind_velo.append(cleaned_wind_velo[row, col, obs])
flat_wind_velo = np.array(flat_wind_velo)

flat_temp_min = []
for obs in range(cleaned_temp_min.shape[2]):
    for row in range(cleaned_temp_min.shape[0]):
        for col in range(cleaned_temp_min.shape[1]):
            flat_temp_min.append(cleaned_temp_min[row, col, obs])
flat_temp_min = np.array(flat_temp_min)

flat_temp_max = []
for obs in range(cleaned_temp_max.shape[2]):
    for row in range(cleaned_temp_max.shape[0]):
        for col in range(cleaned_temp_max.shape[1]):
            flat_temp_max.append(cleaned_temp_max[row, col, obs])
flat_temp_max = np.array(flat_temp_max)

flat_humidity = []
for obs in range(cleaned_humidity.shape[2]):
    for row in range(cleaned_humidity.shape[0]):
        for col in range(cleaned_humidity.shape[1]):
            flat_humidity.append(cleaned_humidity[row, col, obs])
flat_humidity = np.array(flat_humidity)

flat_precipitation = []
for obs in range(cleaned_precipitation.shape[2]):
    for row in range(cleaned_precipitation.shape[0]):
        for col in range(cleaned_precipitation.shape[1]):
            flat_precipitation.append(cleaned_precipitation[row, col, obs])
flat_precipitation = np.array(flat_precipitation)

flat_drought = []
for obs in range(cleaned_drought.shape[2]):
    for row in range(cleaned_drought.shape[0]):
        for col in range(cleaned_drought.shape[1]):
            flat_drought.append(cleaned_drought[row, col, obs])
flat_drought = np.array(flat_drought)

flat_vegetation = []
for obs in range(cleaned_vegetation.shape[2]):
    for row in range(cleaned_vegetation.shape[0]):
        for col in range(cleaned_vegetation.shape[1]):
            flat_vegetation.append(cleaned_vegetation[row, col, obs])
flat_vegetation = np.array(flat_vegetation)

flat_population = []
for obs in range(cleaned_population.shape[2]):
    for row in range(cleaned_population.shape[0]):
        for col in range(cleaned_population.shape[1]):
            flat_population.append(cleaned_population[row, col, obs])
flat_population = np.array(flat_population)

flat_ERC = []
for obs in range(cleaned_ERC.shape[2]):
    for row in range(cleaned_ERC.shape[0]):
        for col in range(cleaned_ERC.shape[1]):
            flat_ERC.append(cleaned_ERC[row, col, obs])
flat_ERC = np.array(flat_ERC)

#print(flat_prev_masks.shape)
#print(flat_avg_nbrs.shape)
X_TRAIN_FEATURES = ['prev_fire', 
                    'euclid_distance', 
                    'elevation', 'wind_dir', 'wind_velo', 'temp_min', 'temp_max', 'humidity', 'precipitation', 'drought', 'vegetation', 'population', 'ERC']
X_train = np.vstack((
    np.transpose(flat_prev_masks), 
    np.transpose(flat_euclid_distance),
    np.transpose(flat_elevation),
    np.transpose(flat_wind_dir),
    np.transpose(flat_wind_velo),
    np.transpose(flat_temp_min),
    np.transpose(flat_temp_max),
    np.transpose(flat_humidity),
    np.transpose(flat_precipitation),
    np.transpose(flat_drought),
    np.transpose(flat_vegetation),
    np.transpose(flat_population),
    np.transpose(flat_ERC)
))

X_train = np.transpose(X_train) #so that observations correspond to rows now

#Vectorize labels
flat_labels = []
for obs in range(certain_labels.shape[2]):
    for row in range(certain_labels.shape[0]):
        for col in range(certain_labels.shape[1]):
            flat_labels.append(certain_labels[row, col, obs])
flat_labels = np.array(flat_labels)

Y_train = flat_labels
#print(Y_train.shape)

variables = ["currently on fire?", "# of neighbors currently on fire"]

from sklearn.model_selection import train_test_split
X_train_r, X_test_r, Y_train_r, Y_test_r = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Assuming X_train and Y_train are numpy arrays of your dataset
# Convert the datasets to PyTorch tensors
X_train_tensor = torch.tensor(X_train_r, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_r, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train_r, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test_r, dtype=torch.float32)

# Assuming the corruption mask is meant to be some kind of dropout mask
corruption_mask = torch.ones_like(X_train_tensor)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the autoencoder model
autoencoder = Autoencoder()

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Move the model and corruption mask to the appropriate device (CPU or GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
autoencoder = autoencoder.to(device)
corruption_mask = corruption_mask.to(device)

num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for data in train_dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = autoencoder(inputs, corruption_mask)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_dataloader)}')

# Optionally, test the model
with torch.no_grad():
    test_loss = 0.0
    for data in test_dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = autoencoder(inputs, corruption_mask)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_dataloader)}')


NotImplementedError: Only 3D, 4D, 5D padding with non-constant padding are supported for now

## Autoencoder v2

In [41]:
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

# Creating a PyTorch class
# 28*28 ==> 9 ==> 28*28
class AE(torch.nn.Module):
	def __init__(self):
		super().__init__()
		
		# Building an linear encoder with Linear
		# layer followed by Relu activation function
		# 784 ==> 9
		self.encoder = torch.nn.Sequential(
			torch.nn.Linear(28 * 28, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 64),
			torch.nn.ReLU(),
			torch.nn.Linear(64, 36),
			torch.nn.ReLU(),
			torch.nn.Linear(36, 18),
			torch.nn.ReLU(),
			torch.nn.Linear(18, 9)
		)
		
		# Building an linear decoder with Linear
		# layer followed by Relu activation function
		# The Sigmoid activation function
		# outputs the value between 0 and 1
		# 9 ==> 784
		self.decoder = torch.nn.Sequential(
			torch.nn.Linear(9, 18),
			torch.nn.ReLU(),
			torch.nn.Linear(18, 36),
			torch.nn.ReLU(),
			torch.nn.Linear(36, 64),
			torch.nn.ReLU(),
			torch.nn.Linear(64, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 28 * 28),
			torch.nn.Sigmoid()
		)

	def forward(self, x):
		encoded = self.encoder(x)
		decoded = self.decoder(encoded)
		return decoded


## Load

In [51]:
#Vectorize previous fire masks
flat_prev_masks = []

#Remembering that Python is zero-indexed is going to be VERY IMPORTANT in the below!
for obs in range(certain_prev_masks.shape[2]):
    for row in range(certain_prev_masks.shape[0]):
        for col in range(certain_prev_masks.shape[1]):
            flat_prev_masks.append(certain_prev_masks[row, col, obs])
flat_prev_masks = np.array(flat_prev_masks)

#Vectorize euc distance
flat_euclid_distance = []
for obs in range(euclid_distance.shape[2]):
    for row in range(euclid_distance.shape[0]):
        for col in range(euclid_distance.shape[1]):
            flat_euclid_distance.append(euclid_distance[row, col, obs])
flat_euclid_distance = np.array(flat_euclid_distance)

flat_elevation = []
for obs in range(cleaned_elevation.shape[2]):
    for row in range(cleaned_elevation.shape[0]):
        for col in range(cleaned_elevation.shape[1]):
            flat_elevation.append(cleaned_elevation[row, col, obs])
flat_elevation = np.array(flat_elevation)

flat_wind_dir = []
for obs in range(cleaned_wind_dir.shape[2]):
    for row in range(cleaned_wind_dir.shape[0]):
        for col in range(cleaned_wind_dir.shape[1]):
            flat_wind_dir.append(cleaned_wind_dir[row, col, obs])
flat_wind_dir = np.array(flat_wind_dir)

flat_wind_velo = []
for obs in range(cleaned_wind_velo.shape[2]):
    for row in range(cleaned_wind_velo.shape[0]):
        for col in range(cleaned_wind_velo.shape[1]):
            flat_wind_velo.append(cleaned_wind_velo[row, col, obs])
flat_wind_velo = np.array(flat_wind_velo)

flat_temp_min = []
for obs in range(cleaned_temp_min.shape[2]):
    for row in range(cleaned_temp_min.shape[0]):
        for col in range(cleaned_temp_min.shape[1]):
            flat_temp_min.append(cleaned_temp_min[row, col, obs])
flat_temp_min = np.array(flat_temp_min)

flat_temp_max = []
for obs in range(cleaned_temp_max.shape[2]):
    for row in range(cleaned_temp_max.shape[0]):
        for col in range(cleaned_temp_max.shape[1]):
            flat_temp_max.append(cleaned_temp_max[row, col, obs])
flat_temp_max = np.array(flat_temp_max)

flat_humidity = []
for obs in range(cleaned_humidity.shape[2]):
    for row in range(cleaned_humidity.shape[0]):
        for col in range(cleaned_humidity.shape[1]):
            flat_humidity.append(cleaned_humidity[row, col, obs])
flat_humidity = np.array(flat_humidity)

flat_precipitation = []
for obs in range(cleaned_precipitation.shape[2]):
    for row in range(cleaned_precipitation.shape[0]):
        for col in range(cleaned_precipitation.shape[1]):
            flat_precipitation.append(cleaned_precipitation[row, col, obs])
flat_precipitation = np.array(flat_precipitation)

flat_drought = []
for obs in range(cleaned_drought.shape[2]):
    for row in range(cleaned_drought.shape[0]):
        for col in range(cleaned_drought.shape[1]):
            flat_drought.append(cleaned_drought[row, col, obs])
flat_drought = np.array(flat_drought)

flat_vegetation = []
for obs in range(cleaned_vegetation.shape[2]):
    for row in range(cleaned_vegetation.shape[0]):
        for col in range(cleaned_vegetation.shape[1]):
            flat_vegetation.append(cleaned_vegetation[row, col, obs])
flat_vegetation = np.array(flat_vegetation)

flat_population = []
for obs in range(cleaned_population.shape[2]):
    for row in range(cleaned_population.shape[0]):
        for col in range(cleaned_population.shape[1]):
            flat_population.append(cleaned_population[row, col, obs])
flat_population = np.array(flat_population)

flat_ERC = []
for obs in range(cleaned_ERC.shape[2]):
    for row in range(cleaned_ERC.shape[0]):
        for col in range(cleaned_ERC.shape[1]):
            flat_ERC.append(cleaned_ERC[row, col, obs])
flat_ERC = np.array(flat_ERC)

#print(flat_prev_masks.shape)
#print(flat_avg_nbrs.shape)
X_TRAIN_FEATURES = ['prev_fire', 
                    'euclid_distance', 
                    'elevation', 'wind_dir', 'wind_velo', 'temp_min', 'temp_max', 'humidity', 'precipitation', 'drought', 'vegetation', 'population', 'ERC']
X_train = np.vstack((
    np.transpose(flat_prev_masks), 
    np.transpose(flat_euclid_distance),
    np.transpose(flat_elevation),
    np.transpose(flat_wind_dir),
    np.transpose(flat_wind_velo),
    np.transpose(flat_temp_min),
    np.transpose(flat_temp_max),
    np.transpose(flat_humidity),
    np.transpose(flat_precipitation),
    np.transpose(flat_drought),
    np.transpose(flat_vegetation),
    np.transpose(flat_population),
    np.transpose(flat_ERC)
))

X_train = np.transpose(X_train) #so that observations correspond to rows now

#Vectorize labels
flat_labels = []
for obs in range(certain_labels.shape[2]):
    for row in range(certain_labels.shape[0]):
        for col in range(certain_labels.shape[1]):
            flat_labels.append(certain_labels[row, col, obs])
flat_labels = np.array(flat_labels)

Y_train = flat_labels
#print(Y_train.shape)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)

# Ensure the dimensions match
print(X_train_tensor.shape, Y_train_tensor.shape)

# Create the TensorDataset
dataset = TensorDataset(X_train_tensor, Y_train_tensor)

# Create the DataLoader
loader = DataLoader(dataset, batch_size=14, shuffle=True)

torch.Size([76800, 13]) torch.Size([76800])


## Initialize

In [52]:
# Model Initialization
model_nn = AE()

# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()

# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model_nn.parameters(),
							lr = 1e-1,
							weight_decay = 1e-8)


## Train

In [55]:
epochs = 20
outputs = []
losses = []
for epoch in range(epochs):
    for (image, _) in loader:
    
        # Reshaping the image to (-1, 784)
        image = image.reshape(-1, 28*28)

        # Output of Autoencoder
        reconstructed = model_nn(image)

        # Calculating the loss function
        loss = loss_function(reconstructed, image)

        # The gradients are set to zero,
        # the gradient is computed and stored.
        # .step() performs parameter update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Storing the losses in a list for plotting
        losses.append(loss)
    outputs.append((epochs, image, reconstructed))

# Defining the Plot Style
plt.style.use('fivethirtyeight')
plt.xlabel('Iterations')
plt.ylabel('Loss')

# Plotting the last 100 values
plt.plot(losses[-100:])


RuntimeError: shape '[-1, 784]' is invalid for input of size 182

## Reconstruct

In [ ]:
for i, item in enumerate(image):

    # Reshape the array for plotting
    item = item.reshape(-1, 28, 28)
    plt.imshow(item[0])

for i, item in enumerate(reconstructed):
    item = item.reshape(-1, 28, 28)
    plt.imshow(item[0])


# Plotting function

Let's plot the data!

First we define the names for each of our variables.

In [ ]:
TITLES = [
  'Elevation',
  'Wind\ndirection',
  'Wind\nvelocity',
  'Min\ntemp',
  'Max\ntemp',
  'Humidity',
  'Precip',
  'Drought',
  'Vegetation',
  'Population\ndensity',
  'Energy\nrelease\ncomponent',
  'Previous\nfire\nmask',
  'Fire\nmask'
]

Define some helper variables for the plot. 

In [ ]:
# Number of rows of data samples to plot
n_rows = 5 
# Number of data variables
n_features = inputs.shape[3]
# Variables for controllong the color map for the fire masks
CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
BOUNDS = [-1, -0.1, 0.001, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)

In [ ]:
fig = plt.figure(figsize=(15,6.5))

for i in range(n_rows):
  for j in range(n_features + 1):
    plt.subplot(n_rows, n_features + 1, i * (n_features + 1) + j + 1)
    if i == 0:
      plt.title(TITLES[j], fontsize=13)
    if j < n_features - 1:
      plt.imshow(inputs[i, :, :, j], cmap='viridis')
    if j == n_features - 1:
      plt.imshow(inputs[i, :, :, -1], cmap=CMAP, norm=NORM)
    if j == n_features:
      plt.imshow(labels[i, :, :, 0], cmap=CMAP, norm=NORM) 
    plt.axis('off')
plt.tight_layout()